In [ ]:
# %pip install pandas
# %pip install numpy
# %pip install fastparquet


In [4]:
import pandas as pd
import numpy as np


In [10]:
import ast

### 1. CARGA DE DATOS

In [ ]:
# Se lee el archivo output_steam_games.json que esta definido como un unico objeto json y se define el dataframe correspondiente

df_games = pd.read_json('./datasets/output_steam_games.json', lines=True)

print(df_games.head())   # Se observan una gran cantidad de valores nulos en la filas iniciales

print(df_games.tail())

print(df_games.shape)


In [9]:
#print(df_games.tail())
print(df_games.shape)

(120445, 13)


In [11]:
def load_json_lines(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            data.append(ast.literal_eval(line))
    return pd.DataFrame(data)

df_reviews = load_json_lines("./datasets/australian_user_reviews.json")
df_items = load_json_lines("./datasets/australian_users_items.json")

In [12]:
# Vista preliminar de los datos
print(df_reviews.head())
#print(df_reviews.tail())
print(df_reviews.shape)

             user_id                                           user_url  \
0  76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1            js41637               http://steamcommunity.com/id/js41637   
2          evcentric             http://steamcommunity.com/id/evcentric   
3              doctr                 http://steamcommunity.com/id/doctr   
4          maplemage             http://steamcommunity.com/id/maplemage   

                                             reviews  
0  [{'funny': '', 'posted': 'Posted November 5, 2...  
1  [{'funny': '', 'posted': 'Posted June 24, 2014...  
2  [{'funny': '', 'posted': 'Posted February 3.',...  
3  [{'funny': '', 'posted': 'Posted October 14, 2...  
4  [{'funny': '3 people found this review funny',...  
(25799, 3)


In [14]:
print(df_items.head())
print(df_items.shape)

             user_id  items_count           steam_id  \
0  76561197970982479          277  76561197970982479   
1            js41637          888  76561198035864385   
2          evcentric          137  76561198007712555   
3         Riot-Punch          328  76561197963445855   
4              doctr          541  76561198002099482   

                                            user_url  \
0  http://steamcommunity.com/profiles/76561197970...   
1               http://steamcommunity.com/id/js41637   
2             http://steamcommunity.com/id/evcentric   
3            http://steamcommunity.com/id/Riot-Punch   
4                 http://steamcommunity.com/id/doctr   

                                               items  
0  [{'item_id': '10', 'item_name': 'Counter-Strik...  
1  [{'item_id': '10', 'item_name': 'Counter-Strik...  
2  [{'item_id': '1200', 'item_name': 'Red Orchest...  
3  [{'item_id': '10', 'item_name': 'Counter-Strik...  
4  [{'item_id': '300', 'item_name': 'Day of Defea..

### 2. ETL-GAMES

In [15]:
print(df_games.shape)
print(df_games.info())      # Se observa una gran cantidad de datos nulos

(120445, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB
None


In [16]:
print(df_games.isnull().sum())

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64


In [17]:
# Se eliminan las filas del dataframe que tienen todos sus valores nulos
df_games = df_games.dropna(how='all')
print(df_games.isnull().sum())

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


In [19]:
# Se eliminan las filas con datos nulos en las columans 'id', 'release_date', 'developer' y 'price'
df_games = df_games.dropna(subset=['id', 'release_date', 'developer', 'price'])
print(df_games.info())

<class 'pandas.core.frame.DataFrame'>
Index: 27596 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22916 non-null  object 
 1   genres        27462 non-null  object 
 2   app_name      27596 non-null  object 
 3   title         27596 non-null  object 
 4   url           27596 non-null  object 
 5   release_date  27596 non-null  object 
 6   tags          27488 non-null  object 
 7   reviews_url   27596 non-null  object 
 8   specs         27316 non-null  object 
 9   price         27596 non-null  object 
 10  early_access  27596 non-null  float64
 11  id            27596 non-null  float64
 12  developer     27596 non-null  object 
dtypes: float64(2), object(11)
memory usage: 2.9+ MB
None


In [ ]:
# Se eliminan las filas con datos nulos en las columans 'id', 'release_date', 'developer' y 'price'
df_games = df_games.dropna(subset=['genres'])
print(df_games.info())

In [ ]:
# Se obtiene el año del release_date requerido para el análisis
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')
df_games['year'] = df_games['release_date'].dt.year


df_games = df_games.dropna(subset=['year'])
df_games['year'] = df_games['year'].astype(int)

print(df_games.head(1))

In [23]:
# Se cambia a letra minuscula todos los valores en genres, requerido para el analisis posterior
df_games['genres'] = df_games['genres'].astype(str).str.lower()

print(df_games['genres'])

In [ ]:
# Se Cambia el nombre de la columna "id" por "item_id" y se convierte a enteros requerido para analisis posterior
df_games = df_games.rename(columns={'id': 'item_id'})
df_games['item_id'] = df_games['item_id'].astype(int) 

print(df_games.info())

In [32]:
print(df_games.columns)

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'item_id',
       'developer', 'year'],
      dtype='object')


In [104]:
# Se genera el dataframe con las columnas requieridas para las funciones a desarrollar
df_games_api=df_games.loc[:, ['genres', 'title', 'item_id', 'year']]

print(df_games_api.head(1))
print(df_games_api.shape)

                                                  genres                title  \
88310  ['action', 'casual', 'indie', 'simulation', 's...  Lost Summoner Kitty   

       item_id    year  
88310   761140  2018.0  
(27596, 4)


In [66]:
df_games_api.to_parquet("./datasets/df_game_api.parquet", index=False)

### 3. ETL-REVIEWS

In [ ]:
print(df_reviews.head())     # Se observa una seria almacenada en la columna 'review' que se debe expandir
#print(df_reviews.tail())
print(df_reviews.shape)

In [28]:
# Se expandira el dataframe df_reviews para que quede una fila por cada recomendación 
# Y una columna por cada dato de la seria almacenada en la columna de 'review'

expanded_rows = []
for index, row in df_reviews.iterrows():
    user_id  = row['user_id']
    user_url = row['user_url']

    review_list = row['reviews']

    for it in review_list:
        # se genera diccionario del nuevo df expandido
        new_item = {    'user_id'  : user_id ,
                        'user_url' : user_url,
                        **it
                    }
        expanded_rows.append(new_item)

df_reviews_exp = pd.DataFrame(expanded_rows)

In [31]:
print(df_reviews_exp.columns)

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'review'],
      dtype='object')


In [37]:
print(df_reviews_exp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB
None


In [40]:
# Reemplazar valores vacíos, 'null' y 'None' con NaN en todo el DataFrame
df_reviews_exp.replace(['', 'null', 'None'], np.nan, inplace=True)

In [42]:
# Se verifican y se eliminan las filas duplicadas
filas_dup = df_reviews_exp.duplicated().sum()
print(filas_dup)

874


In [43]:
print(df_reviews_exp.shape)

(59305, 9)


In [44]:
df_reviews_exp.drop_duplicates(inplace=True)
print(df_reviews_exp.shape)

(58431, 9)


In [ ]:
print(df_reviews_exp.head(5))

In [46]:
# Se obtiene el año del review a partir de la columna 'posted'

# Se convierte 'posted' a  tipo datetime
df_reviews_exp['posted'] = pd.to_datetime(df_reviews_exp['posted'], format='Posted %B %d, %Y.', errors='coerce')

print(df_reviews_exp['posted'].value_counts()) 

posted
2014-06-21    220
2014-06-20    192
2014-06-27    171
2014-06-23    171
2013-12-26    169
             ... 
2011-03-25      1
2011-11-16      1
2011-05-28      1
2012-02-01      1
2012-11-03      1
Name: count, Length: 1644, dtype: int64


In [47]:
# Crea la columna 'year' a partir de 'posted'
df_reviews_exp['year'] = df_reviews_exp['posted'].dt.year.astype('Int64')

print(df_reviews_exp['year'].value_counts()) 

year
2014    21834
2015    18154
2013     6713
2012     1201
2011      530
2010       66
Name: count, dtype: Int64


In [48]:
# Eliminar filas con valores nulos en 'review'
var_nulos = (df_reviews_exp['review'].isnull().sum()) 
print(var_nulos)
print(df_reviews_exp.info())
#df_reviews_exp = df_reviews_exp.dropna(subset=['review'])

30
<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 59304
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      58431 non-null  object        
 1   user_url     58431 non-null  object        
 2   funny        8010 non-null   object        
 3   posted       48498 non-null  datetime64[ns]
 4   last_edited  6037 non-null   object        
 5   item_id      58431 non-null  object        
 6   helpful      58431 non-null  object        
 7   recommend    58431 non-null  bool          
 8   review       58401 non-null  object        
 9   year         48498 non-null  Int64         
dtypes: Int64(1), bool(1), datetime64[ns](1), object(7)
memory usage: 4.6+ MB
None


In [49]:
df_reviews_exp = df_reviews_exp.dropna(subset=['review'])
print(df_reviews_exp.info())


<class 'pandas.core.frame.DataFrame'>
Index: 58401 entries, 0 to 59304
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      58401 non-null  object        
 1   user_url     58401 non-null  object        
 2   funny        8009 non-null   object        
 3   posted       48471 non-null  datetime64[ns]
 4   last_edited  6037 non-null   object        
 5   item_id      58401 non-null  object        
 6   helpful      58401 non-null  object        
 7   recommend    58401 non-null  bool          
 8   review       58401 non-null  object        
 9   year         48471 non-null  Int64         
dtypes: Int64(1), bool(1), datetime64[ns](1), object(7)
memory usage: 4.6+ MB
None


In [50]:
# Se Eliminan los valores que no tienen un año especifico 
df_reviews_exp = df_reviews_exp.dropna(subset=['year'])
print(df_reviews_exp.info())


<class 'pandas.core.frame.DataFrame'>
Index: 48471 entries, 0 to 59276
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      48471 non-null  object        
 1   user_url     48471 non-null  object        
 2   funny        5696 non-null   object        
 3   posted       48471 non-null  datetime64[ns]
 4   last_edited  5057 non-null   object        
 5   item_id      48471 non-null  object        
 6   helpful      48471 non-null  object        
 7   recommend    48471 non-null  bool          
 8   review       48471 non-null  object        
 9   year         48471 non-null  Int64         
dtypes: Int64(1), bool(1), datetime64[ns](1), object(7)
memory usage: 3.8+ MB
None


In [65]:
# Se genera el dataframe con las columnas requieridas para las funciones a desarrollar
df_reviews_api=df_reviews_exp.loc[:, ['user_id', 'item_id', 'recommend', 'review', 'year']]

print(df_reviews_api.head())
print(df_reviews_api.shape)

             user_id item_id  recommend  \
0  76561197970982479    1250       True   
1  76561197970982479   22200       True   
2  76561197970982479   43110       True   
3            js41637  251610       True   
4            js41637  227300       True   

                                              review  year  
0  Simple yet with great replayability. In my opi...  2011  
1               It's unique and worth a playthrough.  2011  
2  Great atmosphere. The gunplay can be a bit chu...  2011  
3  I know what you think when you see this title ...  2014  
4  For a simple (it's actually not all that simpl...  2013  
(48471, 5)


In [52]:
# Convertir los datos de la columna 'item_id' a formato entero
df_reviews_api['item_id'] = df_reviews_api['item_id'].astype(int) 
print(df_reviews_api.info())

<class 'pandas.core.frame.DataFrame'>
Index: 48471 entries, 0 to 59276
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  48471 non-null  object
 1   item_id  48471 non-null  int32 
 2   review   48471 non-null  object
 3   year     48471 non-null  Int64 
dtypes: Int64(1), int32(1), object(2)
memory usage: 1.7+ MB
None


In [71]:
df_reviews_api.to_parquet("./datasets/df_reviews_api.parquet", index=False)

### 4. ETL-ITEMS

In [54]:
#print(df_items.head())
print(df_items.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB
None


In [73]:
# Se expandira el dataframe df_items para que quede una fila por cada recomendación 
# Y una columna por cada dato de la seria almacenada en la columna de 'items'

expanded_rows = []
for index, row in df_items.iterrows():
    user_id  = row['user_id']
    user_url = row['user_url']

    items_list = row['items']

    for it in items_list:
        # se genera diccionario del nuevo df expandido
        new_item = {    'user_id'  : user_id ,
                        'user_url' : user_url,
                        **it
                    }
        expanded_rows.append(new_item)

df_items_exp = pd.DataFrame(expanded_rows)

In [74]:
print(df_items_exp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   user_url          object
 2   item_id           object
 3   item_name         object
 4   playtime_forever  int64 
 5   playtime_2weeks   int64 
dtypes: int64(2), object(4)
memory usage: 235.9+ MB
None


In [75]:
# Se eliminan las filas duplicadas
filas_dup = df_items_exp.duplicated().sum()
print(filas_dup)
print(df_items_exp.shape)

59104
(5153209, 6)


In [76]:
df_items_exp.drop_duplicates(inplace=True)
print(df_items_exp.shape)

(5094105, 6)


In [77]:
# Se eliminan las columnas que nos son utiles para nuestro estudio
df_items_api = df_items_exp.drop(['user_url', 'item_name', 'playtime_2weeks'], axis=1)
df_items_api.columns

Index(['user_id', 'item_id', 'playtime_forever'], dtype='object')

In [78]:
# Convertir los datos de la columna 'item_id' a entero
df_items_api['item_id'] = df_items_api['item_id'].astype(int)
print(df_items_api.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5094105 entries, 0 to 5153208
Data columns (total 3 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           int32 
 2   playtime_forever  int64 
dtypes: int32(1), int64(1), object(1)
memory usage: 136.0+ MB
None


In [79]:
# Para ver las filas con valores de playtime_forever=0 (para considerar eliminar de encontrarse necesario) 
conteo_valores = df_items_api['playtime_forever'].value_counts()

In [80]:
print(conteo_valores)

playtime_forever
0        1847730
1         101586
2          34391
3          31530
4          29127
          ...   
76541          1
64676          1
53328          1
44029          1
34753          1
Name: count, Length: 48861, dtype: int64


In [85]:
df_items_api = df_items_api.loc[df_items_api['playtime_forever'] != 0]
conteo_valores = df_items_api['playtime_forever'].value_counts()
print(conteo_valores)

playtime_forever
1        101586
2         34391
3         31530
4         29127
5         27612
          ...  
76541         1
64676         1
53328         1
44029         1
34753         1
Name: count, Length: 48860, dtype: int64


In [64]:
filas_nulas = df_items_api['playtime_forever'].isnull().sum()
print (filas_nulas)

0


In [86]:
df_items_api.to_parquet("./datasets/df_items_api.parquet", index=False)

### 5. Analisis de Sentimiento

In [88]:
#%pip install langdetect 
from langdetect import detect

In [92]:
#%pip install nltk
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [89]:
# Determinar el lenguaje en el que fueron escritos los 'review'

def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return None

# Aplicar la función para detectar idioma y crear una nueva columna 'idioma'
df_reviews_api['idioma'] = df_reviews_api['review'].apply(detectar_idioma)

# Calcular el conteo y porcentaje de cada idioma
conteo_por_idioma = df_reviews_api['idioma'].value_counts()
porcentaje_por_idioma = df_reviews_api['idioma'].value_counts(normalize=True) * 100

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_idiomas = resumen_idiomas.sort_values(by='Conteo', ascending=False)
resumen_idiomas.head()


,Conteo,Porcentaje
idioma,,
en,37225,77.5%
pt,1812,3.77%
es,1058,2.2%
de,981,2.04%
so,872,1.82%


In [90]:
# Para eliminar los review que no estan en Ingles 'en' y se almacenara en el dataframe df_reviews_en 
df_reviews_api_en = df_reviews_api[df_reviews_api['idioma']== 'en'].copy()

print(df_reviews_api_en['idioma'].value_counts()) 


idioma
en    37225
Name: count, dtype: int64


In [ ]:
print(df_reviews_api_en.head(1))

In [94]:

# Descargar el lexicon necesario para el análisis de sentimientos
nltk.download('vader_lexicon')

# Crear una instancia del analizador de sentimientos
sia = SentimentIntensityAnalyzer()

# Convertimos la columna 'review' en tipo string para aplicar nuestro modelo de analisis de sentimiento
df_reviews_api_en['review'] = df_reviews_api_en['review'].astype(str)

# Calcula el puntaje de sentimiento para cada reseña y agrega al df una nueva columna "sentimiento"
df_reviews_api_en['sentimiento'] = df_reviews_api_en['review'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Definimos umbrales para clasificar las reseñas en positivas, neutrales y negativas
umbral_positivo = 0.3
umbral_negativo = -0.3

# Crea una nueva columna "sentimiento_etiqueta" para clasificar en positivo, neutral y negativo
df_reviews_api_en['sentimiento_etiqueta'] = df_reviews_api_en['sentimiento'].apply(lambda x: 2 if x > umbral_positivo else (0 if x < umbral_negativo else 1))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Diego\AppData\Roaming\nltk_data...


In [97]:
#print(df_reviews_api_en.head(2))
print(df_reviews_api_en['sentimiento_etiqueta'].value_counts())


sentimiento_etiqueta
2    24571
1     7673
0     4981
Name: count, dtype: int64


In [98]:
print(df_reviews_api_en.columns)

Index(['user_id', 'item_id', 'recommend', 'review', 'year', 'idioma',
       'sentimiento', 'sentimiento_etiqueta'],
      dtype='object')


In [ ]:
df_reviews_api_en = df_reviews_api_en.drop(['idioma', 'review', 'sentimiento'], axis=1)

In [101]:
print(df_reviews_api_en.columns)

Index(['user_id', 'item_id', 'recommend', 'year', 'sentimiento_etiqueta'], dtype='object')


In [102]:
df_reviews_api_en.to_parquet("./datasets/df_reviews_api_en.parquet", index=False)